In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls

gdrive	sample_data


In [71]:
## Import necessary libraries
import numpy as np
import pandas as pd
import os
import seaborn as sns
from sklearn.model_selection import train_test_split ## For splitting data into train and validation
from sklearn.preprocessing import LabelEncoder ## For label encoding(converting categorical values to label)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
os.chdir('/content/gdrive/My Drive/Novartis')

In [5]:
!ls

sample_submission.csv  Test.csv  Train.csv  xgboost.csv


In [6]:
train_data=pd.read_csv('Train.csv')

In [7]:
test_data=pd.read_csv('Test.csv')

In [8]:
print(train_data.shape)
print(test_data.shape)

(23856, 18)
(15903, 17)


In [9]:
train_data.head(3)

,INCIDENT_ID,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,MULTIPLE_OFFENSE
0,CR_102659,04-JUL-04,0,36,34,2,1,5,6,1,6,1,174,1.0,92,29,36,0
1,CR_189752,18-JUL-17,1,37,37,0,0,11,17,1,6,1,236,1.0,103,142,34,1
2,CR_184637,15-MAR-17,0,3,2,3,5,1,0,2,3,1,174,1.0,110,93,34,1


In [ ]:
train_data.tail(3)

,INCIDENT_ID,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,MULTIPLE_OFFENSE
23853,CR_157556,03-APR-12,0,25,25,9,0,3,5,1,6,1,174,0.0,10,29,18,1
23854,CR_103180,25-JAN-04,0,39,39,6,5,2,7,1,6,1,127,0.0,112,103,43,1
23855,CR_22575,08-NOV-94,7,36,34,2,1,9,8,0,5,1,249,1.0,92,29,34,1


In [ ]:
test_data.head(3)

,INCIDENT_ID,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,CR_195453,01-FEB-18,0,30,35,7,3,6,4,0,5,1,174,NaN,72,119,23
1,CR_103520,05-MAR-04,0,44,44,1,3,7,1,4,6,1,316,0.0,12,29,34
2,CR_196089,27-JAN-18,0,34,33,3,5,2,7,3,0,1,316,1.0,72,0,34


In [ ]:
test_data.tail(3)

,INCIDENT_ID,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
15900,CR_115946,22-APR-06,0,26,27,9,0,6,4,2,6,1,0,1.0,72,62,34
15901,CR_137663,03-APR-09,0,21,23,4,1,2,7,1,6,2,249,2.0,92,62,34
15902,CR_33545,24-APR-96,0,4,4,6,5,4,2,5,6,1,249,1.0,72,29,34


## Understanding the data

In [ ]:
def understand_data(data) :
    return(pd.DataFrame({"Datatype":data.dtypes,"No of NAs":data.isna().sum(),
                         "No of Levels":data.apply(lambda x: x.nunique(),axis=0),"Levels":data.apply(lambda x: str(x.unique()),axis=0)}))

In [ ]:
understand_data(train_data)

,Datatype,No of NAs,No of Levels,Levels
INCIDENT_ID,object,0,23856,['CR_102659' 'CR_189752' 'CR_184637' ... 'CR_1...
DATE,object,0,9121,['04-JUL-04' '18-JUL-17' '15-MAR-17' ... '14-S...
X_1,int64,0,8,[0 1 5 7 3 4 2 6]
X_2,int64,0,52,[36 37 3 33 45 30 8 49 4 7 25 21 47 22 15 16 1...
X_3,int64,0,52,[34 37 2 32 45 35 7 49 4 8 25 23 48 22 14 15 1...
X_4,int64,0,10,[2 0 3 10 7 6 9 4 1 5]
X_5,int64,0,5,[1 0 5 3 2]
X_6,int64,0,19,[5 11 1 7 8 9 15 2 4 13 3 16 6 18 12 17 14 10 19]
X_7,int64,0,19,[6 17 0 1 3 8 10 7 2 18 5 12 4 13 16 11 14 15 9]
X_8,int64,0,24,[1 2 0 3 4 5 7 6 12 10 8 20 11 15 9 99 14 13 3...


In [ ]:
understand_data(test_data)

,Datatype,No of NAs,No of Levels,Levels
INCIDENT_ID,object,0,15903,['CR_195453' 'CR_103520' 'CR_196089' ... 'CR_1...
DATE,object,0,7939,['01-FEB-18' '05-MAR-04' '27-JAN-18' ... '07-M...
X_1,int64,0,8,[0 7 1 5 3 4 2 6]
X_2,int64,0,52,[30 44 34 3 7 47 33 21 36 16 4 26 5 49 6 9 25 ...
X_3,int64,0,52,[35 44 33 2 8 48 32 23 34 15 4 27 5 49 6 10 25...
X_4,int64,0,10,[7 1 3 2 4 0 6 9 10 5]
X_5,int64,0,5,[3 5 1 0 2]
X_6,int64,0,19,[6 7 2 9 4 5 1 3 16 15 11 18 8 12 14 17 13 19 10]
X_7,int64,0,19,[4 1 7 8 2 6 0 5 12 10 17 13 3 16 14 11 18 9 15]
X_8,int64,0,22,[0 4 3 1 2 12 5 6 40 7 8 20 11 10 25 9 18 30 1...


In [10]:
## set index for train and test
train_data.set_index('INCIDENT_ID',inplace=True)
test_data.set_index('INCIDENT_ID',inplace=True)

In [11]:
train_data.head()

,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,MULTIPLE_OFFENSE
INCIDENT_ID,,,,,,,,,,,,,,,,,
CR_102659,04-JUL-04,0,36,34,2,1,5,6,1,6,1,174,1.0,92,29,36,0
CR_189752,18-JUL-17,1,37,37,0,0,11,17,1,6,1,236,1.0,103,142,34,1
CR_184637,15-MAR-17,0,3,2,3,5,1,0,2,3,1,174,1.0,110,93,34,1
CR_139071,13-FEB-09,0,33,32,2,1,7,1,1,6,1,249,1.0,72,29,34,1
CR_109335,13-APR-05,0,33,32,2,1,8,3,0,5,1,174,0.0,112,29,43,1


In [ ]:
test_data.head()

,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
INCIDENT_ID,,,,,,,,,,,,,,,,
CR_195453,01-FEB-18,0,30,35,7,3,6,4,0,5,1,174,NaN,72,119,23
CR_103520,05-MAR-04,0,44,44,1,3,7,1,4,6,1,316,0.0,12,29,34
CR_196089,27-JAN-18,0,34,33,3,5,2,7,3,0,1,316,1.0,72,0,34
CR_112195,18-AUG-06,7,3,2,3,5,9,8,0,5,1,174,1.0,112,87,34
CR_149832,31-OCT-11,0,7,8,7,3,2,7,1,5,1,174,0.0,112,93,43


In [ ]:
train_data.dtypes

DATE                 object
X_1                   int64
X_2                   int64
X_3                   int64
X_4                   int64
X_5                   int64
X_6                   int64
X_7                   int64
X_8                   int64
X_9                   int64
X_10                  int64
X_11                  int64
X_12                float64
X_13                  int64
X_14                  int64
X_15                  int64
MULTIPLE_OFFENSE      int64
dtype: object

In [12]:
### Feature Extraction
train_data['date']=pd.to_datetime(train_data['DATE'])

In [13]:
test_data['date']=pd.to_datetime(test_data['DATE'])

In [14]:
## Extract date from date in train_data
train_data['date']= [d.date() for d in train_data['date']]

In [15]:
## Extract date from date in test_data
test_data['date']= [d.date() for d in test_data['date']]

In [16]:
## Drop DATE column both in train and test data as we have extracted features from it
train_data.drop(['DATE'], axis=1, inplace=True)
test_data.drop(['DATE'], axis=1, inplace=True)

In [17]:
### Extract day, month and year from the date column 
train_data['year'] = train_data['date'].apply(lambda x: x.year)
train_data['month'] = train_data['date'].apply(lambda x: x.month)
train_data['day'] = train_data['date'].apply(lambda x: x.day)

In [18]:
### Extract day, month and year from the date column
test_data['year'] = test_data['date'].apply(lambda x: x.year)
test_data['month'] = test_data['date'].apply(lambda x: x.month)
test_data['day'] = test_data['date'].apply(lambda x: x.day)

In [19]:
## drop date column once feature extraction is done from train_data
train_data.drop(['date'],axis=1,inplace=True)

In [20]:
## drop date column once feature extraction is done from test_data
test_data.drop(['date'],axis=1,inplace=True)

In [21]:
train_data.head(1)

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,MULTIPLE_OFFENSE,year,month,day
INCIDENT_ID,,,,,,,,,,,,,,,,,,,
CR_102659,0,36,34,2,1,5,6,1,6,1,174,1.0,92,29,36,0,2004,7,4


In [22]:
test_data.head(1)

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,year,month,day
INCIDENT_ID,,,,,,,,,,,,,,,,,,
CR_195453,0,30,35,7,3,6,4,0,5,1,174,NaN,72,119,23,2018,2,1


In [23]:
## fill NA values
train_data['X_12'].fillna(0,inplace=True)
test_data['X_12'].fillna(0,inplace=True)

In [24]:
train_data.isna().sum()

X_1                 0
X_2                 0
X_3                 0
X_4                 0
X_5                 0
X_6                 0
X_7                 0
X_8                 0
X_9                 0
X_10                0
X_11                0
X_12                0
X_13                0
X_14                0
X_15                0
MULTIPLE_OFFENSE    0
year                0
month               0
day                 0
dtype: int64

In [25]:
test_data.isna().sum()

X_1      0
X_2      0
X_3      0
X_4      0
X_5      0
X_6      0
X_7      0
X_8      0
X_9      0
X_10     0
X_11     0
X_12     0
X_13     0
X_14     0
X_15     0
year     0
month    0
day      0
dtype: int64

In [ ]:
train_data.dtypes

X_1                   int64
X_2                   int64
X_3                   int64
X_4                   int64
X_5                   int64
X_6                   int64
X_7                   int64
X_8                   int64
X_9                   int64
X_10                  int64
X_11                  int64
X_12                float64
X_13                  int64
X_14                  int64
X_15                  int64
MULTIPLE_OFFENSE      int64
year                  int64
month                 int64
day                   int64
dtype: object

In [26]:
## Data type conversions
train_data['X_12']=train_data['X_12'].astype('int64')
test_data['X_12']=test_data['X_12'].astype('int64')

In [32]:
## splitting train data into both train and validation data
X_train,X_validation,y_train,y_validation = train_test_split(train_data.drop('MULTIPLE_OFFENSE',axis=1),train_data['MULTIPLE_OFFENSE'],
                                                             test_size=0.2,random_state=1234)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_validation.shape)
print(y_validation.shape)

In [34]:
## Instantiate label encoder
le_year = LabelEncoder()
le_month = LabelEncoder()
le_day = LabelEncoder()
le_multiple_offense = LabelEncoder()

In [35]:
## Label encoding for train_data
X_train['year'] = le_year.fit_transform(X_train['year'])
X_train['month'] = le_month.fit_transform(X_train['month'])
X_train['day'] = le_day.fit_transform(X_train['day'])

In [37]:
## Label encoding for validation_data
X_validation['year'] = le_year.transform(X_validation['year'])
X_validation['month'] = le_month.transform(X_validation['month'])
X_validation['day'] = le_day.transform(X_validation['day'])

In [38]:
## Do labe encoding for test data 
test_data['year'] = le_year.transform(test_data['year'])
test_data['month'] = le_month.transform(test_data['month'])
test_data['day'] = le_day.transform(test_data['day'])

In [40]:
##Label encoding for the target variable
y_train = le_multiple_offense.fit_transform(y_train)

In [41]:
y_validation = le_multiple_offense.transform(y_validation)

In [42]:
from xgboost import XGBClassifier ## XG boost model
from sklearn.model_selection import GridSearchCV ##For Grid search(cross validation)
from sklearn.metrics import accuracy_score ## For getting accuracy value
from sklearn.metrics import confusion_matrix,classification_report ## For classifier metrics(accuracy,TPR,TNR)

In [ ]:
xgb = XGBClassifier() ## Instantiate XGBClassifier model

optimization_dict = {'max_depth': [2,3,4,5,6,7], ## trying with different max_depth,n_estimators to find best model
                      'n_estimators': [50,60,70,80,90,100,150,200]} 

## Build best model with Grid Search params
model = GridSearchCV(xgb, ## XGB model
                     optimization_dict, ## dictory with different max_depth,n_estimators
                     scoring='accuracy', ## on which parameter we are interested
                     verbose=1, ## for messaging purpose
                     n_jobs=-1) ## Number of jobs to run in parallel. ''-1' means use all processors

%time model.fit(X_train, y_train) ## Fit a model
print(model.best_score_) ## Display best score calues
print(model.best_params_) ## Display best parameters

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  3.9min finished


CPU times: user 5.25 s, sys: 165 ms, total: 5.41 s
Wall time: 3min 54s
0.9992663572267727
{'max_depth': 4, 'n_estimators': 200}


In [43]:
model = XGBClassifier(max_depth=4, n_estimators=200,learning_rate = 0.001,
                      booster ='gbtree',random_state=1234)
%time model.fit(X_train, y_train)

CPU times: user 2.7 s, sys: 11.9 ms, total: 2.71 s
Wall time: 2.81 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.001, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1234,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [44]:
## Get the predictions on train data
train_pred = model.predict(X_train)

In [45]:
## Get the predictions on validation data
validation_pred = model.predict(X_validation)

In [46]:
## confusion matrix for train data
confusion_matrix_train = confusion_matrix(y_train, train_pred)
print(confusion_matrix_train)

[[  585   285]
 [   35 18179]]


In [47]:
# confusion matrix for validation data
confusion_matrix_validation = confusion_matrix(y_validation, validation_pred)
print(confusion_matrix_validation)

[[ 130   68]
 [   8 4566]]


In [48]:
##Calculating the recall for train data
recall_train=np.diag(confusion_matrix_train)/np.sum(confusion_matrix_train,axis=1)
recall_train
np.mean(recall_train)

0.8352460971666358

In [49]:
##Calculating the recall for validation data
recall_validation=np.diag(confusion_matrix_validation)/np.sum(confusion_matrix_validation,axis=1)
recall_validation
np.mean(recall_validation)

0.8274083201936284

In [ ]:
temp=test_data.copy()

In [ ]:
## Get the predictions on test data
y_pred = model.predict(temp)

In [ ]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
temp1 = temp.copy()

In [ ]:
temp1.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,year,month,day
INCIDENT_ID,,,,,,,,,,,,,,,,,,
CR_195453,0,30,35,7,3,6,4,0,5,1,174,0,72,119,23,27,1,0
CR_103520,0,44,44,1,3,7,1,4,6,1,316,0,12,29,34,13,2,4
CR_196089,0,34,33,3,5,2,7,3,0,1,316,1,72,0,34,27,0,26
CR_112195,7,3,2,3,5,9,8,0,5,1,174,1,112,87,34,15,7,17
CR_149832,0,7,8,7,3,2,7,1,5,1,174,0,112,93,43,20,9,30


In [ ]:
temp1['MULTIPLE_OFFENSE'] = le_multiple_offense.inverse_transform(y_pred)

In [ ]:
temp1.reset_index(inplace=True)

In [ ]:
to_submit_1 = temp1[['INCIDENT_ID', 'MULTIPLE_OFFENSE']]

In [ ]:
to_submit_1.MULTIPLE_OFFENSE.value_counts()

1    15365
0      538
Name: MULTIPLE_OFFENSE, dtype: int64

In [ ]:
to_submit_1.shape

(15903, 2)

In [ ]:
test_data.shape

(15903, 18)

In [ ]:
## store to_submit into csv file with name xgboost 
to_submit_1.to_csv('xgboost.csv',index = False)

## Improvement 1

In [50]:
model = XGBClassifier(max_depth=10, n_estimators=200,learning_rate = 0.001,
                      booster ='gbtree',random_state=1234)
%time model.fit(X_train, y_train)

CPU times: user 4.57 s, sys: 7.9 ms, total: 4.57 s
Wall time: 4.58 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.001, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1234,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [51]:
## Get the predictions on train data
train_pred = model.predict(X_train)

In [52]:
## Get the predictions on validation data
validation_pred = model.predict(X_validation)

In [53]:
## confusion matrix for train data
confusion_matrix_train = confusion_matrix(y_train, train_pred)
print(confusion_matrix_train)

[[  859    11]
 [   16 18198]]


In [54]:
# confusion matrix for validation data
confusion_matrix_validation = confusion_matrix(y_validation, validation_pred)
print(confusion_matrix_validation)

[[ 191    7]
 [   7 4567]]


In [55]:
##Calculating the recall for train data
recall_train=np.diag(confusion_matrix_train)/np.sum(confusion_matrix_train,axis=1)
recall_train
np.mean(recall_train)

0.9932389383434999

In [56]:
##Calculating the recall for validation data
recall_validation=np.diag(confusion_matrix_validation)/np.sum(confusion_matrix_validation,axis=1)
recall_validation
np.mean(recall_validation)

0.9815580377451825

In [57]:
temp=test_data.copy()

In [58]:
## Get the predictions on test data
y_pred = model.predict(temp)

In [59]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [60]:
temp1 = temp.copy()

In [61]:
temp1.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,year,month,day
INCIDENT_ID,,,,,,,,,,,,,,,,,,
CR_195453,0,30,35,7,3,6,4,0,5,1,174,0,72,119,23,27,1,0
CR_103520,0,44,44,1,3,7,1,4,6,1,316,0,12,29,34,13,2,4
CR_196089,0,34,33,3,5,2,7,3,0,1,316,1,72,0,34,27,0,26
CR_112195,7,3,2,3,5,9,8,0,5,1,174,1,112,87,34,15,7,17
CR_149832,0,7,8,7,3,2,7,1,5,1,174,0,112,93,43,20,9,30


In [62]:
temp1['MULTIPLE_OFFENSE'] = le_multiple_offense.inverse_transform(y_pred)

In [63]:
temp1.reset_index(inplace=True)

In [64]:
to_submit_2 = temp1[['INCIDENT_ID', 'MULTIPLE_OFFENSE']]

In [65]:
to_submit_2.MULTIPLE_OFFENSE.value_counts()

1    15181
0      722
Name: MULTIPLE_OFFENSE, dtype: int64

In [66]:
to_submit_2.shape

(15903, 2)

In [67]:
test_data.shape

(15903, 18)

In [68]:
## store to_submit into csv file with name xgboost 
to_submit_2.to_csv('xgboost_improvement.csv',index = False)